In [44]:
import bs4 as bs
import requests
import regex as re
import pandas as pd

Constants

In [ ]:
def magic():
    response = get_data()  # This makes the external call to the internet. This is why we want to keep it separate
    data = parse_html(response.text)  # This function can be unittest. It extracts data from response
    save_data(data)  # Possibly, working with files, therefore might need to extract to separate function

In [45]:
URL_SOURCE = "https://www.goodreads.com/"
URL_START = "https://www.goodreads.com/list/tag/"
BOOK_CATEGORIES = [
    "romance",
    "fiction",
    "young-adult",
    "fantasy",
    "science-fiction",
    "non-fiction",
    "children",
    "history",
    "covers",
    "mystery",
    "horror",
    "best",
    "historical-fiction",
    "gay",
    "paranormal",
    "love",
    "titles",
    "contemporary",
    "middle-grade",
    "historical-romance",
    "biography",
    "thriller",
    "series",
    "women",
    "nonfiction",
    "classics",
    "lgbt",
    "graphic-novels",
    "memoir",
    "queer",
]

Auxiliary functions

In [46]:
def get_page_body(url: str):
    try:
        response = requests.get(url, timeout=10)
    except requests.exceptions.HTTPError as errh:
        print ("http error:",errh)
    except requests.exceptions.ConnectionError as errc:
        print ("conection error:",errc)
    except requests.exceptions.Timeout as errt:
        print ("timeout error:",errt)
    except requests.exceptions.RequestException as err:
        print ("other error:",err)
        
    if response.status_code == 200:
        page = bs.BeautifulSoup(response.text)
        return page.body
    else:
        return None
    
def clean_text(s: str):
    s = re.sub(r'[\n\t]', ' ', s)
    s = s.strip()
    s = ' '.join(s.split())
    return s

    
def get_category_urls(input_url: str = URL_START, top_n:int = 1) -> dict:
    category_urls = {}
    for category in BOOK_CATEGORIES:
        page_body = get_page_body(input_url + category)
        if page_body:
            link = page_body.find("div",{"class": "listImgs"}).find("a")['href']
            links = ["https://www.goodreads.com/" + link + f"?page={i}" for i in range(1,top_n+1)]
            category_urls[category] = links
    return category_urls


def get_links(body):
    hrefs = body.find("div",{"class": "listImgs"}).find("a")['href']
    links = ["https://www.goodreads.com/" + hrefs + f"?page={i}" for i in range(1, top_n+1)]

def get_category_urls(input_url: str = URL_START, top_n:int = 1) -> dict:
    category_urls = {}
    {category:get_links_for_body(body) for category in BOOK_CATEGORIES}
    for category in BOOK_CATEGORIES:
        page_body = get_page_body(input_url + category)
        if page_body:
            link = page_body.find("div",{"class": "listImgs"}).find("a")['href']
            links = [f"https://www.goodreads.com/{link}?page={i}" for i in range(1, top_n+1)]
            category_urls[category] = links
    return category_urls


def get_separate_book_urls(url: str):
    page_body = get_page_body(url)
    parse_for_urls(response.text)
    return urls


def get_text(x):
    return clean_text(getattr(x, "text", ""))


def get_book_info(page_body, book_category: str):
    book_info = {}
    book_info["category"] = book_category

    patterns = [
        ["title", "h1", "id", "bookTitle"],
        ['author', 'span', "itemprop", 'name'],
    ]

    for key, el, prop, prop_value in patterns:
        book_info[key] = get_text(page_body.find(el, **{prop:prop_value}))


    book_info["title"] = get_text(page_body.find("h1", id = "bookTitle"))
    book_info["author"] = get_text(page_body.find("span", itemprop="name"))
    book_info["description"] = get_text(page_body.find("div", id="description"))
    book_info["rating"] =  get_text(page_body.find("span", itemprop="ratingValue"))
    book_info["number_of_pages"] = get_text(page_body.find("span", itemprop="numberOfPages"))
    book_info["url"] = url
    return book_info

Main scraping script

In [47]:
books_db = pd.DataFrame()
category_urls = get_category_urls()

for category in category_urls.keys():
    if category_urls[category]:
        for page_url in category_urls[category]:
            books_urls_list = get_separate_book_urls(page_url)
            if books_urls_list:
                for book_url in books_urls_list:
                    book_info = get_book_info(book_url, category)
                    if book_info:
                        books_db = books_db.append(book_info, ignore_index=True)
books_db.to_parquet("books_info.parquet")

In [ ]:
books_db = pd.DataFrame()
category_urls = get_category_urls()

for category in category_urls.keys():
    url = category_urls[category]
    if not url:
        continue

    for page_url in url:
        books_urls_list = get_separate_book_urls(page_url)
        if not books_urls_list:
            continue

        for book_url in books_urls_list:
            book_info = get_book_info(book_url, category)
            if book_info:
                books_db = books_db.append(book_info, ignore_index=True)
books_db.to_parquet("books_info.parquet")

In [ ]:

page_urls = handle_category()
book_urls = get_book_urls(page_urls)
books_data = get_books_data(book_urls)

def get_book_urls():
    # Get response
    # Parse response
    # Return data

def get_books_data():
    # Get response
    # Parse response
    # Return data

books_db = pd.DataFrame(books_data)
books_db.to_parquet("books_info.parquet")

In [ ]:
df = pd.read_parquet("books_info.parquet")

In [1]:
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!
